In [39]:
import pandas as pd
import yahooquery as yq
from yahooquery import Ticker
import matplotlib.pyplot as plt

In [77]:
tickers_argentinos = [
    # "YPFD",  # YPF Sociedad Anónima
    # "GGAL",  # Grupo Financiero Galicia S.A.
    # "BMA",   # Banco Macro S.A.
    # "PAMP",  # Pampa Energía S.A.
    "TECO2", # Telecom Argentina S.A.
    "BBAR",  # BBVA Banco Francés S.A.
    "TGSU2", # Transportadora de Gas del Sur S.A.
    "EDN",   # Empresa Distribuidora y Comercializadora Norte S.A. (Edenor)
    "TRAN",  # Compañía de Transporte de Energía Eléctrica en Alta Tensión Transener S.A.
    "LOMA",  # Loma Negra Compañía Industrial Argentina S.A.
    "ALUA",  # Aluar Aluminio Argentino S.A.I.C.
    "MIRG",  # Mirgor S.A.C.I.F.I.A.
    "CRES",  # Cresud Sociedad Anónima Comercial, Inmobiliaria, Financiera y Agropecuaria
    "SUPV",  # Grupo Supervielle S.A.
    "CEPU",  # Central Puerto S.A.
    "TGN",   # Transportadora de Gas del Norte S.A.
    "METR",  # Metrogas S.A.
    "AUSO",  # Autopistas del Sol S.A.
    "HARG",  # Holcim Argentina S.A.
    "TXAR",  # Ternium Argentina S.A.
    "BYMA",  # Bolsas y Mercados Argentinos S.A.
    "VALO",  # Banco de Valores S.A.
]

In [40]:
def convertir_fecha_a_trimestre(fecha):
    """Convierte una fecha datetime a formato 'YYYY-Qq'."""
    return f"{fecha.year}-Q{(fecha.month - 1) // 3 + 1}"

In [41]:
# Declarar nombre del ticker a trabajar
# simbolo = "GOOG"
# # # Descargar datos históricos
# ticker = Ticker(simbolo)
# ticker.key_stats[simbolo]

In [84]:
def guardar_estados_financieros(simbolo):
    # Crear un archivo Excel con múltiples hojas
    with pd.ExcelWriter(f"{simbolo}_datos_financieros.xlsx") as writer:
        # Balance
        ticker = Ticker(simbolo)
        financial_data = ticker.financial_data[simbolo]

        # Convert financial_data to DataFrame with an index
        financials = pd.DataFrame([financial_data], index=[simbolo])
        financials.to_excel(writer, sheet_name="Financials")

        balance = ticker.balance_sheet().T
        balance.columns = pd.to_datetime(balance.iloc[0], format='%d/%m/%Y %H:%M:%S')
        if pd.api.types.is_datetime64_any_dtype(balance.columns):
            balance.columns = balance.columns.to_period("Q").strftime("%Y-Q%q")
        else:
            balance.columns = pd.to_datetime(balance.columns, errors='coerce').to_period("Q").strftime("%Y-Q%q")
        balance.to_excel(writer, sheet_name="Balance")

        # Estado de resultados
        ingresos = ticker.income_statement().T
        ingresos.columns = pd.to_datetime(ingresos.iloc[0], format='%d/%m/%Y %H:%M:%S')
        ingresos.columns = pd.to_datetime(ingresos.columns).to_period("Q").strftime("%Y-Q%q")
        ingresos.to_excel(writer, sheet_name="Estado_Resultados")

        # Flujo de efectivo
        flujo = ticker.cash_flow().T
        flujo.columns = pd.to_datetime(flujo.iloc[0], format='%d/%m/%Y %H:%M:%S')
        flujo.columns = pd.to_datetime(flujo.columns).to_period("Q").strftime("%Y-Q%q")
        flujo.to_excel(writer, sheet_name="Flujo_Efectivo")

        historial = ticker.history(period="max")
        historial.to_excel(writer, sheet_name="Historial")

        # Información adicional
        info = ticker.asset_profile
        pd.DataFrame([info]).to_excel(writer, sheet_name="Información")

    print(f"Datos financieros de {simbolo} guardados exitosamente")

In [ ]:
# Declarar nombre del ticker a trabajar
# simbolo = "GOOG"
# # # Descargar datos históricos
# ticker = Ticker(simbolo)
ticker.key_stats[simbolo]

In [85]:
# guardar_estados_financieros("YPF")
for ticker in tickers_argentinos:
    try:
        guardar_estados_financieros(ticker)
    except Exception as e:
        print(f"Error al guardar datos financieros para {ticker}: {e}")

C:\Users\Carlo\.virtualenvs\lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")


IndexError: At least one sheet must be visible

In [13]:
# Visualizar cambio del balance de la empresa a lo largo del tiempo
balance = ticker.balancesheet
balance = balance.dropna(axis=1, how="all")
balance = balance.dropna(axis=0, how="all")
balance = balance.T # Transponer para visualización
balance

429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TSLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TSLA&crumb=Edge%3A+Too+Many+Requests


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def obtener_y_graficar_balances(ticker_symbol="TSLA", valores=None):
    """
    Obtiene datos de balance de yfinance y genera gráficos de tarta comparativos.

    Args:
        ticker_symbol: El símbolo del ticker (ej. "TSLA").
        valores: Lista de valores a graficar.
    """

    if valores is None:
        valores = [
            "Treasury Shares Number",
            "Ordinary Shares Number",
            "Share Issued",
            "Total Debt",
            "Tangible Book Value",
            "Invested Capital",
            "Working Capital",
            "Net Tangible Assets",
            "Capital Lease Obligations",
            "Common Stock Equity"
        ]

    stock = Ticker(ticker_symbol)

    try:
        balance_sheet = stock.balance_sheet
    except Exception as e:
        print(f"Error al obtener el balance: {e}")
        return

    if balance_sheet.empty:
        print("No se encontraron datos de balance.")
        return

    # Obtener los dos últimos años disponibles
    years = balance_sheet.columns[:2]
    if len(years) < 2:
        print("No hay suficientes datos para comparar dos años.")
        return

    fig, axes = plt.subplots(1, 2, figsize=(15, 7))  # Dos subplots uno al lado del otro
    fig.suptitle(f"Comparación de Balances de {ticker_symbol} ({years[0].year} vs {years[1].year})", fontsize=16)


    for i, year in enumerate(years):
        data = balance_sheet[year].loc[valores].dropna() # Seleccionar valores y quitar NaN
        if data.empty: # Comprobar si hay datos para graficar
            axes[i].set_title(f"Sin datos disponibles para {year.year}")
            continue

        axes[i].pie(data, labels=data.index, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 8})
        axes[i].set_title(f"Balance {year.year}")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar layout para evitar superposición de títulos
    plt.show()

In [12]:
obtener_y_graficar_balances()

# Ejemplo con valores específicos:
valores_especificos = ["Total Debt", "Common Stock Equity", "Working Capital"]
obtener_y_graficar_balances(valores=valores_especificos)


C:\Users\Carlo\.virtualenvs\lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
C:\Users\Carlo\.virtualenvs\lib\site-packages\yahooquery\ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)
C:\Users\Carlo\.virtualenvs\lib\site-packages\yahooquery\ticker.py:1335: FutureWarning: A value is trying to be set 

open        high         low  \
symbol date                                                            
GOOG   2020-03-25                  56.323502   57.445000   54.300499   
       2020-03-26                  55.590000   58.498501   54.676498   
       2020-03-27                  56.283501   57.533501   55.295502   
       2020-03-30                  56.251999   57.581501   54.824001   
       2020-03-31                  57.365002   58.765499   56.907001   
...                                      ...         ...         ...   
       2025-03-18                 165.960007  166.440002  158.800003   
       2025-03-19                 163.914993  168.132996  163.050003   
       2025-03-20                 163.824997  167.029999  163.139999   
       2025-03-21                 163.380005  166.470001  163.029999   
       2025-03-24 16:00:01-04:00  169.264999  170.330002  167.440002   

                                       close    volume    adjclose  dividends  \
symbol date                                                                     
GOOG   2020-03-25                  55.124500  81630000   54.864639        0.0   
       2020-03-26                  58.087502  71434000   57.813671        0.0   
       2020-03-27                  55.535500  64170000   55.273693        0.0   
       2020-03-30                  57.341000  51482000   57.070690        0.0   
       2020-03-31                  58.140499  49728000   57.866417        0.0   
...                                      ...       ...         ...        ...   
       2025-03-18                 162.669998  24616800  162.669998        0.0   
       2025-03-19                 166.279999  24955700  166.279999        0.0   
       2025-03-20                 165.050003  19981500  165.050003        0.0   
       2025-03-21                 166.250000  29849000  166.250000        0.0   
       2025-03-24 16:00:01-04:00  169.929993  18667259  169.929993        0.0   

                                  splits  
symbol date                               
GOOG   2020-03-25                    0.0  
       2020-03-26                    0.0  
       2020-03-27                    0.0  
       2020-03-30                    0.0  
       2020-03-31                    0.0  
...                                  ...  
       2025-03-18                    0.0  
       2025-03-19                    0.0  
       2025-03-20                    0.0  
       2025-03-21                    0.0  
       2025-03-24 16:00:01-04:00     0.0  

[1256 rows x 8 columns]